## rent information with distance
- ../data/curated/rent_distance.csv
- rent_index (int), SA2 (int), rent (float), bedroom (int), baths (int), parking (int), Latitude (float), Longitude (float), school_dis (float), station_dis (float)


In [1]:
import geopandas as gpd
import pandas as pd
from openrouteservice import client
import folium
import numpy as np
from folium.plugins import BeautifyIcon
import fontawesome as fa
from shapely.geometry import Polygon, LineString, Point
import time
from shapely.ops import cascaded_union

INPUT_PATH = f"../data/raw/"
# GIT
TOKEN = "5b3ce3597851110001cf62483f8783dee87f49d1a08c14865546e9d5"
# UNIMELB
# TOKEN = "5b3ce3597851110001cf62488e7bed0411524218994d57e759d00421"
ors = client.Client(key=TOKEN)
SLEEP_TIME = 3.1

params_route = {'profile': 'driving-car',
            'format_out': 'geojson',
            'geometry': 'true',
            'format': 'geojson',
            'instructions': 'false'
            }


In [12]:
def find_k_nearest(point, from_gdf, k):
    """
        Find closed point to {point} from  {from_gdf}.geometry, return index in from_gdf
        coor: Point,  coordinate of the start point
        from_gdf: geopandas dataframe to search for closed point
    """
    distance = from_gdf["geometry"].distance(point).to_list()
    index = from_gdf.index.to_list()
    index_len = len(index)
    output = sorted([[distance[i], int(index[i])] for i in range(index_len)])
    
    if k > len(output):
        return [output[i][1] for i in range(len(output))]
    else:
        return [output[i][1] for i in range(k)]

def point_wise_distance(point_from, point_to, params_route = {
            'profile': 'driving-car',
            'format_out': 'geojson',
            'geometry': 'true',
            'format': 'geojson',
            'instructions': 'false'
            }):
    """
        Use OpenRouteService to request information about travelling
            from {point_from} to {point_to} usingparams_route
        point_from: Point object to travel from
        point_to: Point object to travel to
    """
    if not point_from or not point_to: return 0
    time.sleep(SLEEP_TIME)
    print(f"requesting distance from {[point_from.x, point_from.y]} to {[point_to.x, point_to.y]}")
    params_route['coordinates'] = [[point_from.x, point_from.y], [point_to.x, point_to.y]]
    
    try:
        json = ors.directions(**params_route)
        distance = json['features'][0]['properties']['summary']['distance']
        return distance
    except Exception as e:
        print(f"\n\n{e}\n\n")
        return 0





In [3]:
point_a = Point(144.29047, -36.76841)
point_b = Point(144.30797, -36.76838)
json_route = point_wise_distance(point_a, point_b, params_route)

requesting distance from [144.29047, -36.76841] to [144.30797, -36.76838]


#### Load files

In [134]:
# read / prepare rent data
rent_gdf = pd.read_csv(f"../data/curated/rent.csv").drop(["geometry"], axis=1).dropna()
rent_gdf = gpd.GeoDataFrame(rent_gdf, geometry=gpd\
            .points_from_xy(rent_gdf["Longitude"], rent_gdf["Latitude"]))
rent_gdf["geometry"] = rent_gdf["geometry"].set_crs("epsg:7844")
# rent_gdf["geometry"] = rent_gdf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# feature seleciton on rent
rent_gdf = rent_gdf[["rent_id", "SA2", "geometry"]].set_index("rent_id")

print(rent_gdf.head(2))
print(rent_gdf.shape)
print(rent_gdf.dtypes)

               SA2                     geometry
rent_id                                        
0        206021110  POINT (143.79387 -37.56307)
1        206051134  POINT (143.77011 -37.54724)
(3748, 2)
SA2            int64
geometry    geometry
dtype: object


In [102]:
# read local files
boundary_gdf = gpd.read_file(f"{INPUT_PATH}ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp").dropna()
school_gdf = pd.read_csv(f"{INPUT_PATH}ACARA/School_location.csv")
school_gdf = gpd.GeoDataFrame(school_gdf, geometry=gpd\
            .points_from_xy(school_gdf["Longitude"], school_gdf["Latitude"])).dropna()
school_gdf["geometry"] = school_gdf["geometry"].set_crs("epsg:7844")

# feature selction on boundary gdf
boundary_gdf = boundary_gdf.loc[boundary_gdf["STE_NAME21"] == "Victoria"]
boundary_gdf = boundary_gdf[["SA2_CODE21", "geometry"]]
boundary_gdf["SA2_CODE21"] = boundary_gdf["SA2_CODE21"].astype("int64")

# feature seleciton on school gdf
school_gdf = school_gdf[["school_id", "SA2", "geometry"]].set_index("school_id")
school_gdf["SA2"] = school_gdf["SA2"].astype("int64")
school_gdf = school_gdf.drop_duplicates()

print(boundary_gdf.head(2))
print(boundary_gdf.shape)
print(boundary_gdf.dtypes)

print(school_gdf.head(2))
print(school_gdf.shape)
print(school_gdf.dtypes)

     SA2_CODE21                                           geometry
644   201011001  POLYGON ((143.78282 -37.56666, 143.75558 -37.5...
645   201011002  POLYGON ((143.81896 -37.55582, 143.81644 -37.5...
(522, 2)
SA2_CODE21       int64
geometry      geometry
dtype: object
                 SA2                     geometry
school_id                                        
0          201011001  POINT (143.80174 -37.55497)
1          201011001  POINT (143.79995 -37.55349)
(2269, 2)
SA2            int64
geometry    geometry
dtype: object


In [136]:
# rent_gdf = rent_gdf.loc[rent_gdf["SA2"] == 202011020]
# boundary_gdf = boundary_gdf.loc[boundary_gdf["SA2_CODE21"] == 202011020]
# school_gdf = school_gdf.loc[school_gdf["SA2"] == 202011020]

# print(rent_gdf.head())
# print(boundary_gdf.head())
# print(school_gdf.head())

#### Find school distance

In [137]:
point_class = type(Point(0, 1))

rent_gdf["geometry"] = rent_gdf["geometry"].apply(lambda x: x if x.is_valid else None)
rent_gdf = rent_gdf.dropna()

print(rent_gdf.shape)


(3748, 2)


In [18]:
rent_gdf["tar_point"] = rent_gdf["geometry"].apply(lambda x: school_gdf.loc[find_k_nearest(x, school_gdf, 1)]["geometry"].to_list()[0])
rent_gdf["tar_distan"] = 0

print(rent_gdf.dtypes)

# points_from_xy(rent_gdf["Longitude"], rent_gdf["Latitude"]))

print(rent_gdf.head())
print(rent_gdf.columns)
print(rent_gdf.shape)
rent_gdf["tar_lo"] = rent_gdf["tar_point"].apply(lambda x: x.x)
rent_gdf["tar_la"] = rent_gdf["tar_point"].apply(lambda x: x.y)
rent_gdf = rent_gdf.drop("tar_point", axis=1)
rent_gdf.to_file(filename="../data/curated/rent_school.shp", index="rent_id")

#### Request distance

In [63]:
# rent_gdf = gpd.read_file("../data/curated/rent_school.shp")
# rent_gdf = rent_gdf.set_index("rent_id")
# rent_gdf["tar_point"] = rent_gdf.apply(lambda x: Point(x["tar_lo"], x["tar_la"]), axis=1)
# index = [2867, 2971, 3249, 3269, 3279, 3344]
# print(start+gap*i, start+gap*(1+i))

# print(rent_gdf.shape)

# ## feature enginnering
# rent_gdf.loc[index, "tar_distan"] = rent_gdf.loc[index].apply(lambda x: point_wise_distance(x["geometry"], x["tar_point"]), axis=1)


# rent_gdf = rent_gdf.drop("tar_point", axis=1)
# rent_gdf.to_file("../data/curated/rent_school.shp")
# rent_gdf[["SA2", "tar_distan"]].to_csv("../data/curated/rent_school.csv")

# print(f"distance_pulled: from {start+gap*i} to {start+gap*(1+i)}")
# print(rent_gdf.loc[rent_gdf["tar_distan"]!=0.0].shape)

In [16]:
rent_df = gpd.read_file("../data/curated/rent_school.shp")
print(rent_df.loc[rent_df["tar_distan"]!=0.0].tail())
print(rent_df.shape)
print(rent_df.loc[rent_df["tar_distan"]==0.0].shape)
print(rent_df.loc[rent_df["tar_distan"]!=0.0].shape)


      rent_id        SA2  tar_distan      tar_lo     tar_la  \
3743     3743  213041573       282.7  142.481482 -38.373613   
3744     3744  203021040       931.0  142.456146 -38.378018   
3745     3745  203021485      1306.5  142.497399 -38.389498   
3746     3746  206011495       459.2  142.519533 -38.385247   
3747     3747  212031458       253.6  142.477992 -38.378988   

                         geometry  
3743  POINT (142.48063 -38.37525)  
3744  POINT (142.46395 -38.38286)  
3745  POINT (142.49286 -38.38424)  
3746  POINT (142.51554 -38.38682)  
3747  POINT (142.47632 -38.38010)  
(3748, 6)
(0, 6)
(3748, 6)


In [18]:
### last save: low: 2200, upper: 3350

## read rent file
# ['features'][0]['properties']['summary']['distance'])
start = 3350
gap = 50
for i in range(8):
    rent_gdf = gpd.read_file("../data/curated/rent_school.shp")
    rent_gdf = rent_gdf.set_index("rent_id")
    rent_gdf["tar_point"] = rent_gdf.apply(lambda x: Point(x["tar_lo"], x["tar_la"]), axis=1)

    if(start+gap*i > len(rent_gdf.index.to_list())): break
    if(start+gap*(1+i) > len(rent_gdf.index.to_list())): 
        index = rent_gdf.index.to_list()[start+gap*i : len(rent_gdf.index.to_list())]
    else:
        index = rent_gdf.index.to_list()[start+gap*i : start+gap*(1+i)]
    print(start+gap*i, start+gap*(1+i))

    print(rent_gdf.shape)

    ## feature enginnering
    rent_gdf.loc[index, "tar_distan"] = rent_gdf.loc[index].apply(lambda x: point_wise_distance(x["geometry"], x["tar_point"]), axis=1)


    rent_gdf = rent_gdf.drop("tar_point", axis=1)
    rent_gdf.to_file("../data/curated/rent_school.shp")
    rent_gdf[["SA2", "tar_distan"]].to_csv("../data/curated/rent_school.csv")

    print(f"distance_pulled: from {start+gap*i} to {start+gap*(1+i)}")
    print(rent_gdf.loc[rent_gdf["tar_distan"]!=0.0].shape)
    

In [10]:
# rent_gdf = rent_gdf.drop("tar_point", axis=1)
rent_gdf.to_file("../data/curated/rent_school.shp")
rent_df = gpd.read_file("../data/curated/rent_school.shp")
print(rent_df.loc[rent_df["tar_distan"]!=0.0].shape)

(3748, 6)


## PTV station

In [129]:
ptv_dir = "../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/"
## read data
metro_train_gdf = gpd.read_file(f"{ptv_dir}PTV_METRO_TRAIN_STATION.shp")
region_train_gdf = gpd.read_file(f"{ptv_dir}PTV_REGIONAL_TRAIN_STATION.shp")
boundary_gdf = gpd.read_file(f"{INPUT_PATH}ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp").dropna()

## feature selection on boundary data
boundary_gdf = boundary_gdf.loc[boundary_gdf["STE_NAME21"] == "Victoria"]
boundary_gdf["SA2_CODE21"] = boundary_gdf["SA2_CODE21"].astype("int64")

## feature selection on station gdf
metro_train_gdf = metro_train_gdf[["STOP_ID", "geometry"]]
region_train_gdf = region_train_gdf[["STOP_ID", "geometry"]]
polygons = boundary_gdf[boundary_gdf["SA2_NAME21"].str.contains("Melbourne CBD")]["geometry"].to_list()
cbd = gpd.GeoSeries(cascaded_union(polygons))[0].centroid

## convate station data
station_gdf = gpd.GeoDataFrame(pd.concat([metro_train_gdf,region_train_gdf])).dropna().drop_duplicates().set_index("STOP_ID")
station_gdf.loc[0] = cbd
station_gdf["geometry"] = station_gdf["geometry"].set_crs("epsg:7844")
station_gdf.index = station_gdf.index.map(int)

print(station_gdf.shape)
print(station_gdf.tail())

(331, 1)
                            geometry
STOP_ID                             
47647    POINT (144.60874 -37.87287)
47648    POINT (144.69472 -37.83215)
48804    POINT (144.60411 -37.71253)
52011    POINT (144.73575 -37.76613)
0        POINT (144.96257 -37.81361)


/tmp/ipykernel_561/1645532818.py:15: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  cbd = gpd.GeoSeries(cascaded_union(polygons))[0].centroid


In [130]:
# read / prepare rent data
rent_gdf = pd.read_csv(f"../data/curated/rent.csv").drop(["geometry"], axis=1).dropna()
rent_gdf = gpd.GeoDataFrame(rent_gdf, geometry=gpd\
            .points_from_xy(rent_gdf["Longitude"], rent_gdf["Latitude"]))
rent_gdf["geometry"] = rent_gdf["geometry"].set_crs("epsg:7844")

# feature seleciton on rent
rent_gdf = rent_gdf[["rent_id", "SA2", "geometry"]].set_index("rent_id")
print(station_gdf.head(2))
print(station_gdf.shape)
print(station_gdf.dtypes)

print(rent_gdf.head(2))
print(rent_gdf.shape)
print(rent_gdf.dtypes)

                            geometry
STOP_ID                             
19970    POINT (144.95231 -37.78118)
19971    POINT (144.93933 -37.78813)
(331, 1)
geometry    geometry
dtype: object
               SA2                     geometry
rent_id                                        
0        206021110  POINT (143.79387 -37.56307)
1        206051134  POINT (143.77011 -37.54724)
(3748, 2)
SA2            int64
geometry    geometry
dtype: object


In [73]:
print(station_gdf.index)

Int64Index([19970, 19971, 19972, 19973, 19974, 19975, 19976, 19977, 19978,
            19979,
            ...
            44953, 45918, 47482, 47641, 47642, 47647, 47648, 48804, 52011,
                0],
           dtype='int64', name='STOP_ID', length=331)


In [77]:
rent_gdf["tar_point"] = rent_gdf["geometry"].apply(lambda x: station_gdf.loc[find_k_nearest(x, station_gdf, 1)]["geometry"].to_list()[0])

rent_gdf["tar_distan"] = 0

print(rent_gdf.dtypes)

# points_from_xy(rent_gdf["Longitude"], rent_gdf["Latitude"]))

print(rent_gdf.head())
print(rent_gdf.columns)
print(rent_gdf.shape)
rent_gdf["tar_lo"] = rent_gdf["tar_point"].apply(lambda x: x.x)
rent_gdf["tar_la"] = rent_gdf["tar_point"].apply(lambda x: x.y)
rent_gdf = rent_gdf.drop("tar_point", axis=1)
rent_gdf.to_file(filename="../data/curated/rent_station.shp", index="rent_id")

               SA2                     geometry  tar_distan      tar_lo  \
rent_id                                                                   
0        206021110  POINT (143.79387 -37.56307)           0  143.820074   
1        206051134  POINT (143.77011 -37.54724)           0  143.820074   
2        206051514  POINT (143.80033 -37.56632)           0  143.820074   
3        206071517  POINT (143.78949 -37.56345)           0  143.820074   
4        207011520  POINT (143.78604 -37.55055)           0  143.820074   

            tar_la  
rent_id             
0       -37.539703  
1       -37.539703  
2       -37.539703  
3       -37.539703  
4       -37.539703  
Index(['SA2', 'geometry', 'tar_distan', 'tar_lo', 'tar_la'], dtype='object')
(3748, 5)


# RUN THIS

In [11]:
rent_gdf = gpd.read_file("../data/curated/rent_station.shp")
rent_gdf = rent_gdf.set_index("rent_id")
rent_gdf["tar_point"] = rent_gdf.apply(lambda x: Point(x["tar_lo"], x["tar_la"]), axis=1)

index = [3671]

print(rent_gdf.shape)

## feature enginnering
rent_gdf.loc[index, "tar_distan"] = rent_gdf.loc[index].apply(lambda x: point_wise_distance(x["geometry"], x["tar_point"]), axis=1)


rent_gdf = rent_gdf.drop("tar_point", axis=1)
rent_gdf.to_file("../data/curated/rent_station.shp")
rent_gdf[["SA2", "tar_distan"]].to_csv("../data/curated/rent_station.csv")

/home/bruce/.venv/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


(3748, 6)
requesting distance from [142.0336453, -36.7882751] to [142.20056163846513, -36.707487637483]


404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 142.0336453 -36.7882751.'}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663390866404}})




In [14]:
rent_df = gpd.read_file("../data/curated/rent_station.shp")
print(rent_df.loc[rent_df["tar_distan"]!=0.0].tail())
print(rent_df.shape)
print(rent_df.loc[rent_df["tar_distan"]==0.0].shape)
print(rent_df.loc[rent_df["tar_distan"]!=0.0].shape)
print(rent_df.loc[(rent_df["tar_distan"]==0.0) & (rent_df["rent_id"] <= 3500)].index)

      rent_id        SA2  tar_distan      tar_lo     tar_la  \
3743     3743  213041573      1397.1  142.475551 -38.385001   
3744     3744  203021040      1365.4  142.475551 -38.385001   
3745     3745  203021485      2163.9  142.475551 -38.385001   
3746     3746  206011495      4168.1  142.538877 -38.386379   
3747     3747  212031458       875.8  142.475551 -38.385001   

                         geometry  
3743  POINT (142.48063 -38.37525)  
3744  POINT (142.46395 -38.38286)  
3745  POINT (142.49286 -38.38424)  
3746  POINT (142.51554 -38.38682)  
3747  POINT (142.47632 -38.38010)  
(3748, 6)
(0, 6)
(3748, 6)
Int64Index([], dtype='int64')


In [15]:
# ### last save: low: 1500, upper: 3500

# ## read rent file
# # ['features'][0]['properties']['summary']['distance'])
start = 3500
gap = 50
for i in range(5):
    rent_gdf = gpd.read_file("../data/curated/rent_station.shp")
    rent_gdf = rent_gdf.set_index("rent_id")
    rent_gdf["tar_point"] = rent_gdf.apply(lambda x: Point(x["tar_lo"], x["tar_la"]), axis=1)

    if(start+gap*i > len(rent_gdf.index.to_list())): break
    if(start+gap*(1+i) > len(rent_gdf.index.to_list())): 
        index = rent_gdf.index.to_list()[start+gap*i : len(rent_gdf.index.to_list())]
    else:
        index = rent_gdf.index.to_list()[start+gap*i : start+gap*(1+i)]
    print(start+gap*i, start+gap*(1+i))

    print(rent_gdf.shape)

    ## feature enginnering
    rent_gdf.loc[index, "tar_distan"] = rent_gdf.loc[index].apply(lambda x: point_wise_distance(x["geometry"], x["tar_point"]), axis=1)


    rent_gdf = rent_gdf.drop("tar_point", axis=1)
    rent_gdf.to_file("../data/curated/rent_station.shp")
    rent_gdf[["SA2", "tar_distan"]].to_csv("../data/curated/rent_station.csv")

    print(f"distance_pulled: from {start+gap*i} to {start+gap*(1+i)}")
    print(rent_gdf.loc[rent_gdf["tar_distan"]!=0.0].shape)
    

In [16]:
# rent_gdf = rent_gdf.drop("tar_point", axis=1)
rent_gdf.to_file("../data/curated/rent_station_bac.shp")
rent_df = gpd.read_file("../data/curated/rent_station_bac.shp")
print(rent_df.loc[rent_df["tar_distan"]!=0.0].shape)

(3748, 6)


## Combine distance

In [51]:
## read files
rent_df = pd.read_csv("../data/curated/rent.csv", index_col="rent_id")
schooldis_gdf = gpd.read_file("../data/curated/rent_school.shp")
stationdis_gdf = gpd.read_file("../data/curated/rent_station.shp")
boundary_gdf = gpd.read_file("../data/raw/ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp")

## feature selection
schooldis_gdf = schooldis_gdf.set_index("rent_id")
schooldis_gdf = schooldis_gdf[["tar_distan"]].rename(columns={"tar_distan": "school_dis"})
stationdis_gdf = stationdis_gdf.set_index("rent_id")
stationdis_gdf = stationdis_gdf[["tar_distan"]].rename(columns={"tar_distan": "station_dis"})


# print(rent_df.head())
print(schooldis_gdf.head())
print(stationdis_gdf.head())

         school_dis
rent_id            
0            1651.7
1            1249.5
2            2094.0
3            3988.4
4            1120.8
         station_dis
rent_id             
0             5895.5
1             7529.9
2             6864.2
3             7111.3
4             6272.3


In [81]:
## match distances
rentdis_df = rent_df.join(schooldis_gdf, how="inner")
rentdis_df = rentdis_df.join(stationdis_gdf, how="inner")

## recheck SA2
rentdis_gdf = gpd.GeoDataFrame(rentdis_df, geometry=gpd\
        .points_from_xy(rentdis_df["Longitude"], rentdis_df["Latitude"])).set_crs("EPSG:7844")
rentdis_gdf  = gpd.sjoin(rentdis_gdf, boundary_gdf, how="right")
rentdis_gdf = rentdis_gdf[['index_left', 'SA2_CODE21', 'rent', 'bedroom', 'baths', 'parking', 'Latitude',
       'Longitude', 'school_dis', 'station_dis']]
rentdis_gdf = rentdis_gdf.dropna().rename(columns={"index_left": "rent_index", "SA2_CODE21": "SA2"})
rentdis_gdf = rentdis_gdf.astype({
        "rent_index": int, 
        "SA2": int,
        'bedroom': int,
        'baths': int,
        'parking': int})
rentdis_gdf = rentdis_gdf.set_index("rent_index")

pd.DataFrame(rentdis_gdf).to_csv("../data/curated/rent_distance.csv")
print(rentdis_gdf.shape)
print(rentdis_gdf.head())

(3748, 9)
                  SA2   rent  bedroom  baths  parking   Latitude   Longitude  \
rent_index                                                                     
0           201011001  490.0        4      2        2 -37.563073  143.793875   
1           201011001  420.0        4      2        2 -37.547241  143.770106   
2           201011001  520.0        4      2        2 -37.566319  143.800328   
3           201011001  440.0        4      2        2 -37.563453  143.789489   
4           201011001  440.0        4      2        2 -37.550549  143.786038   

            school_dis  station_dis  
rent_index                           
0               1651.7       5895.5  
1               1249.5       7529.9  
2               2094.0       6864.2  
3               3988.4       7111.3  
4               1120.8       6272.3  


## Plot

In [108]:
print(boundary_gdf.dtypes)

SA2_CODE21       int64
SA2_NAME21      object
CHG_FLAG21      object
CHG_LBL21       object
SA3_CODE21      object
SA3_NAME21      object
SA4_CODE21      object
SA4_NAME21      object
GCC_CODE21      object
GCC_NAME21      object
STE_CODE21      object
STE_NAME21      object
AUS_CODE21      object
AUS_NAME21      object
AREASQKM21     float64
LOCI_URI21      object
geometry      geometry
dtype: object


In [7]:
## icon: pen, car, cart-shopping
rent_gdf = gpd.read_file("../data/curated/rent_station.shp")
rent_gdf = rent_gdf.set_index("rent_id")
rent_gdf["tar_point"] = rent_gdf.apply(lambda x: Point(x["tar_lo"], x["tar_la"]), axis=1)
print(rent_gdf.dtypes)

rent_df = rent_gdf.loc[range(5)]
# station_df = station_gdf

print(rent_df.shape)
print(rent_df.head())


base_map = folium.Map(location=[-37.53970, 143.82007],
    tiles="Stamen Terrain", zoom_start=12)
# geoJSON = boundary_df[["SA2_CODE21", "geometry"]]\
#     .drop_duplicates("SA2_CODE21").to_json()
# base_map.add_child(folium.Choropleth(geo_data=geoJSON,
#                                     name="choropleth"))

for coord in zip(rent_df["geometry"].y, rent_df["geometry"].x):
    base_map.add_child(folium.Marker(location=coord, icon=folium.Icon(color='lightgray',
                                       icon_color='#cc0000',
                                       icon='home',
                                       prefix='fa',
                                       )))

for coord in zip(rent_df["tar_point"].y, rent_df["tar_point"].x):
    base_map.add_child(folium.Marker(location=coord, icon=folium.Icon(color='white',
                                       icon_color='#1a1aff',
                                       icon='pen',
                                       prefix='fa',
                                       )))

base_map

SA2              int64
tar_distan     float64
tar_lo         float64
tar_la         float64
geometry      geometry
tar_point     geometry
dtype: object
(5, 6)
               SA2  tar_distan      tar_lo     tar_la  \
rent_id                                                 
0        206021110      5895.5  143.820074 -37.539703   
1        206051134      7529.9  143.820074 -37.539703   
2        206051514      6864.2  143.820074 -37.539703   
3        206071517      7111.3  143.820074 -37.539703   
4        207011520      6272.3  143.820074 -37.539703   

                            geometry                    tar_point  
rent_id                                                            
0        POINT (143.79387 -37.56307)  POINT (143.82007 -37.53970)  
1        POINT (143.77011 -37.54724)  POINT (143.82007 -37.53970)  
2        POINT (143.80033 -37.56632)  POINT (143.82007 -37.53970)  
3        POINT (143.78949 -37.56345)  POINT (143.82007 -37.53970)  
4        POINT (143.78604 -37.550

/home/bruce/.venv/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [115]:
params_route = {'profile': 'driving-car',
                'format_out': 'geojson',
                'geometry': 'true',
                'format': 'geojson',
                'instructions': 'false',
                'coordinates': [[144.29047, -36.76841], [144.30797, -36.76838]]
                }

json_route = ors.directions(**params_route)



# POINT (144.29047 -36.76841)  202011020
# 1  POINT (144.29047 -36.76841)  202011020
# 2  POINT (144.29047 -36.76841)  202011020
# 3  POINT (144.29047 -36.76841)  202011020
# 4  POINT (144.29047 -36.76841)  202011020
#      SA2_CODE21                                           geometry
# 665   202011020  POLYGON ((144.29049 -36.76159, 144.29048 -36.7...
#            SA2                     geometry
# 107  202011020  POINT (144.30797 -36.76838)
# 108  202011020  POINT (144.29822 -36.76643)
# 109  202011020  POINT (144.31346 -36.78263)
# 110  202011020  POINT (144.29099 -36.76552)
# 111  202011020  POINT (144.31124 -36.76855)



In [123]:
print(json_route['features'][0]['properties']['summary']['duration'])

188.7


In [134]:
folium.features.GeoJson(json_route).add_to(base_map)

base_map